# Gila River Water Rights - Agricultural NDVI

## How has restoring the water rights of the Akimel O’‘otham and Tohono O’’odham tribes altered agricultural lands in the Gila River region?

Cultivated crops represent a significant amount of area within the Gila River region. Since crops require irrigation, I was curious if the restored water rights had any impact on NDVI when looking only at agricultural land.

Using the [Annual National Land Cover Database (NLCD)](https://www.usgs.gov/centers/eros/science/annual-national-land-cover-database), I compared cropland NDVI between Gila River Tribal Areas and the surrounding non-tribal region.

### Analysis steps
- Step 0: Set up
    - Restore variables from previous analysis notebooks 00-04
- Step 1: Load annual NLCD data
    - Note: I used the https://www.mrlc.gov/viewer/ to download data and uploaded it to codespaces
    - I am only using 2001 and 2022 NLCD classes. I was not able to stack all NLCD year .tiffs without crashing codespace.
- Step 2: Create an agriculture vs. non-agriculture mask using the 'cultivated crops' land use category (82)
- Step 3: Assess change in NDVI for stable agriculture pixels
    - Calculate change in agricultural land use between 2001 and 2022
- Step 4: Assess difference in agricultural NDVI inside and outside the Gila River Tribal Area


## Step 0: Set-up
#### Steps
1. Restore variables
2. Load libraries

In [1]:
# Restore variables
%store -r

Unable to restore variable 'ndvi_diff', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


In [2]:
# Import libraries
from pathlib import Path
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap # color ramps
from matplotlib.patches import Patch # legend for final figure
import rioxarray as rxr
import xarray as xr
import hvplot.pandas
import hvplot.xarray # interactive plotting using raster data
import zipfile

## Step 1: Load in NLCD land class data
### Note: This data was downloaded from https://www.mrlc.gov/viewer/

#### Steps
1. Load and extract .zip files of annual NLCD data
2. Use 2001 NLCD cover classes as a proxy for land use for 2001-2011 and 2022 NLCD classes as a proxy for 2012-2022
3. Plot all NLCD classes for Gila River Region

In [3]:
# Path containing .zip files
zip_dir = Path("/workspaces/data/gilariverdata/")
out_dir = Path("/workspaces/data/gilariverdata/NLCD_extracted_tifs")
out_dir.mkdir(parents=True, exist_ok=True)

# Extract and store only the .tiffs
for zip_path in zip_dir.glob("NLCD_mil1dydofjyxbs.zip"):
    with zipfile.ZipFile(zip_path, 'r') as z:
        for file in z.namelist():
            if file.lower().endswith(".tiff"):
                print(f"Extracting {file} from {zip_path.name}")
                z.extract(file, path=out_dir)

Extracting Annual_NLCD_LndCov_2001_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2002_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2003_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2004_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2005_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2006_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2007_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2008_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2009_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2010_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dydofjyxbs.zip
Extracting Annual_NLCD_LndCov_2011_CU_C1V1_mil1dydofjyxbs.tiff from NLCD_mil1dyd

In [4]:
# Open two tifs (2001 and 2022)

# 2001
nlcd_2001_path = out_dir / "Annual_NLCD_LndCov_2001_CU_C1V1_mil1dydofjyxbs.tiff"
nlcd_2001 = rxr.open_rasterio(nlcd_2001_path, mask_and_scale=True).squeeze()

# 2022
nlcd_2022_path = out_dir / "Annual_NLCD_LndCov_2022_CU_C1V1_mil1dydofjyxbs.tiff"
nlcd_2022 = rxr.open_rasterio(nlcd_2022_path, mask_and_scale=True).squeeze()

In [5]:
# Check CRS of the nlcd dataset
print("NLCD 2001 CRS:")
print(nlcd_2001.rio.crs)
print("\nGila GDF CRS:")
print(gila_gdf.crs)

NLCD 2001 CRS:
PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]

Gila GDF CRS:
EPSG:4269


In [6]:
# Reproject gila_gdf from EPSG:4269 to match NLCD for a quick plot
gila_gdf_reproj = gila_gdf.to_crs(nlcd_2001.rio.crs)

# Check crs
print(f"NLCD CRS: {nlcd_2001.rio.crs}")
print(f"Gila GDF CRS (reprojected): {gila_gdf_reproj.crs}")

NLCD CRS: PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Gila GDF CRS (reprojected): PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAM

In [ ]:
# Plot with boundary
fig, ax = plt.subplots(figsize=(12, 8))
nlcd_2001.plot(ax=ax, cmap='tab20', add_colorbar=True)
gila_gdf_reproj.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2.5)
ax.set_title('NLCD Land Cover Classes - 2001 (Gila River Area)')
plt.tight_layout()
plt.show()

**Step 1 summary**
There are many land use categories for the Gila River Region. I am curious about just agricultural land use, so I will select out band 82 'culitvated crops'.

## Step 2: Create an Ag / non-Ag mask
#### Steps
1. Visualize agriculture vs. non-agriculture mask
2. Reproject NLCD data into NDVI CRS
3. Assess change in number of agricultural land use between 2001 and 2022

In [ ]:
# Create binary agriculture layer (1 = ag, 0 = non-ag)
# 2001
ag_2001 = xr.where(nlcd_2001 == 82, 1, 0)
ag_2001.name = 'agriculture'
# 2022
ag_2022 = xr.where(nlcd_2022 == 82, 1, 0)
ag_2022.name = 'agriculture'


# Create custom colormap: white for non-ag, dark green for ag
colors = ['white', 'darkgreen']
cmap = ListedColormap(colors)

# Plot
# Create binary agriculture layers for 2001 and 2022
ag_2001 = xr.where(nlcd_2001 == 82, 1, 0)
ag_2001.name = 'agriculture'

ag_2022 = xr.where(nlcd_2022 == 82, 1, 0)
ag_2022.name = 'agriculture'

# Create custom colormap
colors = ['white', '#154734']
cmap = ListedColormap(colors)

# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 2001
im1 = ag_2001.plot(ax=axes[0], cmap=cmap, vmin=0, vmax=1, add_colorbar=False)
gila_gdf_reproj.plot(ax=axes[0], edgecolor='black', facecolor='none', linewidth=2)
axes[0].set_title('Agriculture - 2001', fontsize=14)

# Plot 2022
im2 = ag_2022.plot(ax=axes[1], cmap=cmap, vmin=0, vmax=1, add_colorbar=False)
gila_gdf_reproj.plot(ax=axes[1], edgecolor='black', facecolor='none', linewidth=2)
axes[1].set_title('Agriculture - 2022', fontsize=14)

# Add shared colorbar
cbar = fig.colorbar(im1,
                    ax=axes,
                    shrink=0.7, 
                    orientation='horizontal', 
                    pad=0.04, 
                    ticks=[0, 1])
cbar.ax.set_xticklabels(['Non-Agriculture', 'Agriculture (Class 82)'])

plt.show()

**Some takeaways from this figure:**
- The amount of agricultural land has decreased between 2001 and 2022
- The CRS appears to be different than our NDVI data (hence the angle)

In [ ]:
# Reproject agriculture mask to match NDVI
ag_2001_reproj = ag_2001.rio.reproject_match(ndvi_2001_2011)
ag_2022_reproj = ag_2022.rio.reproject_match(ndvi_2012_2022)

In [ ]:
# How many less ag cells do we have 2022 compared to 2001?

# Count cells in NDVI reprojected masks
ag_cells_2001 = (ag_2001_reproj == 1).sum().values
ag_cells_2022 = (ag_2022_reproj == 1).sum().values

print(f"Agriculture cells in 2001: {ag_cells_2001:,}")
print(f"Agriculture cells in 2022: {ag_cells_2022:,}")
print(f"Change: {ag_cells_2022 - ag_cells_2001:+,} cells")

# Calculate percentage change
percent_change = ((ag_cells_2022 - ag_cells_2001) / ag_cells_2001) * 100
print(f"Percent change: {percent_change:+.2f}%")

In [ ]:
# Create agriculture change categories
# 0 = never ag
# 1 = only 2001 ag (lost agriculture)
# 2 = only 2022 ag (gained agriculture)  
# 3 = stable ag (both years)

ag_categories = xr.where(
    (ag_2001_reproj == 1) & (ag_2022_reproj == 1), 3,  # Stable ag
    xr.where(
        (ag_2001_reproj == 1) & (ag_2022_reproj == 0), 1,  # Lost ag (only 2001)
        xr.where(
            (ag_2001_reproj == 0) & (ag_2022_reproj == 1), 2,  # Gained ag (only 2022)
            0  # Never ag
        )
    )
)
ag_categories.name = 'agriculture_change'

# Pixels in each category
print(f"0 (Never Ag): {(ag_categories == 0).sum().values:,} pixels")
print(f"1 (Only 2001 Ag - Lost): {(ag_categories == 1).sum().values:,} pixels")
print(f"2 (Only 2022 Ag - Gained): {(ag_categories == 2).sum().values:,} pixels")
print(f"3 (Stable Ag): {(ag_categories == 3).sum().values:,} pixels")

In [ ]:
# Plot ag change categories

# colormap for ag categories 
ag_colors = ['#f9f8ed', '#ffa27c', '#81b38d', '#2c5e43']
ag_labels = ['Never Ag', 'Only 2001 (Lost)', 'Only 2022 (Gained)', 'Stable Ag']
ag_cmap = ListedColormap(ag_colors)

# Plot
fig, ax = plt.subplots(figsize=(14, 8))
im = ag_categories.plot(ax=ax, 
                        cmap=ag_cmap, 
                        vmin=0, 
                        vmax=3, 
                        add_colorbar=False)
gila_gdf.plot(ax=ax, 
              edgecolor='black', 
              facecolor='none', 
              linewidth=2)

# Custom colorbar with proper labels
cbar = plt.colorbar(im, 
                    ax=ax, 
                    ticks=[0.375, 1.125, 1.875, 2.625], 
                    shrink=0.8)
cbar.ax.set_yticklabels(ag_labels)
cbar.set_label('Agricultural Land Status', fontsize=12)

ax.set_title('Agricultural Land Change Categories (2001 vs 2022)', fontsize=14)
plt.tight_layout()
plt.show()

**Takeways from agricultural land use change**
- Agricultural **loss** outside the Gila River Tribal Area, especially adjacent to SE Phoenix / Chandler
- Agricultural **gain** and stablilization within the Gila River Tribal Area

## Step 3: Assess change in NDVI for stable agriculture pixels
#### Steps
1. Mask 2001-2011 NDVI to 2001 and stable agricultural land use
2. Mask 2012-2022 NDVI to 2022 and stable agricultural land use
2. Mask NDVI difference (2012-2022 - 2001-2011) to stable agricultural land use

In [ ]:
# Mask NDVI for different agriculture categories
# 2001-2011: Show categories 1 (only 2001 ag) and 3 (stable ag)
ndvi_2001_2011_ag = ndvi_2001_2011.where((ag_categories == 1) | (ag_categories == 3))

# 2012-2022: Show categories 2 (only 2022 ag) and 3 (stable ag)
ndvi_2012_2022_ag = ndvi_2012_2022.where((ag_categories == 2) | (ag_categories == 3))

# NDVI diff: Show only category 3 (stable ag)
# Re-calculate here

# Difference
ndvi_diff = (
    ndvi_2012_2022
    - ndvi_2001_2011
)

ndvi_diff_stable_ag = ndvi_diff.where(ag_categories == 3)


In [ ]:
# Create three panel plot
fig, axes = plt.subplots(1, 3, figsize =(12,4))

# First decade mean NDVI, masked to 2001 ag
im1 = ndvi_2001_2011_ag.plot(ax=axes[0], 
                    cmap=plt.cm.PiYG, 
                    vmin=-1,vmax=1,
                    add_colorbar=False
                    )
# Plot boundary
gila_gdf.plot(ax=axes[0], edgecolor='black', 
              facecolor='none', linewidth=2)
axes[0].set_title("Mean NDVI of the Gila River\n2001 - 2011")


# Recent NDVI
ndvi_2012_2022_ag.plot(ax=axes[1], 
                    cmap=plt.cm.PiYG, 
                    vmin=-1,vmax=1,
                    add_colorbar=False)
# Plot boundary
gila_gdf.plot(ax=axes[1], edgecolor='black', 
              facecolor='none', linewidth=2)
axes[1].set_title("Mean NDVI of the Gila River\n2012 - 2022")

# Add in scale for first two plots
cbar1 = fig.colorbar(im1, 
                    ax=axes[:2], 
                    shrink=0.7, 
                    orientation="horizontal", 
                    pad=0.1)
cbar1.set_label("NDVI")

# Difference in NDVI in stable ag regions
im2 = ndvi_diff_stable_ag.plot(ax=axes[2], 
                     cmap=plt.cm.BrBG,
                     add_colorbar=False)
# Plot boundary
gila_gdf.plot(ax=axes[2], edgecolor='black', 
              facecolor='none', linewidth=2)
axes[2].set_title("NDVI Difference\n2001-2011 & 2012-2022")

# Add in scale for difference plots
cbar2 = fig.colorbar(im2, 
                    ax=axes[2], 
                    shrink=0.7, 
                    orientation="horizontal", 
                    pad=0.1)
cbar2.set_label("NDVI Difference")

**Agricultural NDVI difference takeaways**
- NDVI difference appears to net neutral across the two studied decades.
- Inside Gila River Tribal area appears to be + NDVI.

## Step 4: Compare agricultural NDVI differences inside and outside the Gila River Tribal Area
#### Steps
1. Calculate mean annual NDVI inside and outside the Gila Tribal Area
2. Calculate and plot difference in NDVI

In [ ]:
# We have NDVI clipped to inside and outside the Gila River Area from notebook 04
print(ndvi_inside.head(2))
print(ndvi_outside.head(2))

In [ ]:
# Clip these to our stable ag region
ndvi_inside_ag = ndvi_inside.where(ag_categories == 3)
ndvi_outside_ag = ndvi_outside.where(ag_categories == 3)

# Recompute July NDVI for inside and outside ag
# Note: this is 25 May - 28 Aug, so not exactly July

# Inside
july_AgNDVI_inside_df = (
    ndvi_inside_ag
    .groupby(ndvi_inside_ag.date.dt.year)
    .mean(...)
    .NDVI
    .to_dataframe()
    )
print("Mean Ag NDVI Inside\n", july_AgNDVI_inside_df.head())

# Outside
july_AgNDVI_outside_df = (
    ndvi_outside_ag
    .groupby(ndvi_outside_ag.date.dt.year)
    .mean(...)
    .NDVI
    .to_dataframe()
    )
print("Mean Ag NDVI Outside\n", july_AgNDVI_outside_df.head())

In [ ]:
# Join inner and outer Gila Ag NDVI

july_ag_ndvi_df = (july_AgNDVI_inside_df[['NDVI']]
                .join(july_AgNDVI_outside_df[['NDVI']],
                      lsuffix='in', rsuffix='out')
                )

july_ag_ndvi_df

In [ ]:
# Plot inner vs. outer mean NDVI with matplotlib
fig, ax = plt.subplots(figsize=(12, 6))

# Plot inside (green) and outside (red)
july_ag_ndvi_df['NDVIin'].plot(ax=ax, 
                               color='#669a77', 
                               linewidth=2.5, 
                               label='Inside Gila', 
                               marker='s')
july_ag_ndvi_df['NDVIout'].plot(ax=ax, 
                                color='#538ab0', 
                                linewidth=2.5, 
                                label='Outside Gila', 
                                marker='o')

# Customize plot
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Mean Agricultural NDVI', fontsize=12)
ax.set_title('Mean Agricultural NDVI Inside and Outside Gila Tribal Area', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**

In [ ]:
# Find the difference inside and outside the boundary

# Add difference column
july_ag_ndvi_df['difference'] = (july_ag_ndvi_df['NDVIin'] 
                              - july_ag_ndvi_df['NDVIout'])

july_ag_ndvi_df

In [ ]:
# Plot the difference with conditional colors
fig, ax = plt.subplots(figsize=(12, 6))

# Create color rule for > or < 0
dif_colors = ['#7AB68B' if x > 0 else '#F3928A' for x in july_ag_ndvi_df['difference']]

# Bar plot
ax.bar(july_ag_ndvi_df.index, 
       july_ag_ndvi_df['difference'], 
       color=dif_colors, 
       alpha=0.7, 
       edgecolor='black', 
       linewidth=1)

# Add line at zero
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

# Labels and title
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('NDVI Difference (Inside - Outside)', fontsize=12)
ax.set_title('NDVI Disparity Between Gila Tribal Area and Surrounding Region', fontsize=14)

# Add legend
legend_elements = [Patch(facecolor='#7AB68B', 
                         alpha=0.7, 
                         label='Inside > Outside'),
                   Patch(facecolor='#F3928A', 
                         alpha=0.7, 
                         label='Inside < Outside')]
ax.legend(handles=legend_elements, fontsize=11)

ax.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Conclusions
When considering **only** agricultural areas, the Gila Tribal Area has a higher NDVI than the surrounding region between 2001-2022. This generally makes sense, as agriculutral areas would take priority in using whatever water the tribes were allocated before their water rights were restored.

What tells a more clear story of the impact of these restored water rights is the change in land use within the tribal vs. non-tribal area. Agriculutural land use grew in the tribal area, and shrank outside during the 22 year period. This could reflect both greater water availability within Gila, and development pressure from sprawing Pheonix.